# Deep Ensemble For Uncertainty Predictions

# Import

In [ ]:
# system
import sys
import os 
import time

# data
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, balanced_accuracy_score
import itertools

# deep learning 
import torch.backends.cudnn as cudnn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader 
import torchvision.transforms as txf
import kornia.augmentation as k

# custom helpers
import trainer
import metrics
import process
import models

# for formatting
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

# Notebook Toggles

In [ ]:
COMPUTE_DISTRIBUTION = False
MODEL_ZOO = True
MODEL_TYPE = ["RES", "DENSE"][0]

## Hyperparameters 

In [ ]:
device = torch.device('cuda:0')
epochs = 30
learning_rate = 3e-4
weight_decay = 1e-5
step_period = 3
lr_decay = 0.95
num_workers = 6
batch_size = 128

# Data

Data is provided by "HAM10000" - 7 classes!

"https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T"

    - AKIEC : Actinic keratoses and intraepithelial carcinoma / Bowen's disease 
    - BCC   : Basal cell carcinoma 
    - BKL.  : Benign keratosis-like lesions
    - DF.   : Dermatofibroma 
    - MEL.  : Melanoma
    - NV.   : Melanocytic nevi 
    - VASC  : Vascular lesions 

## Loading and Statistics

In [ ]:
def toPath(root, image_id):
    return os.path.join(root, image_id + '.npy')

def toLabel(key, mapping):
    return mapping[key]

### HAM

In [ ]:
# paths
ham_path = "YOUR PATH HERE"

# parse data
metadata = pd.read_csv('./data/HAM/metadata.csv')
headers = metadata.head()

# verify data
label_mapping = {label : idx for idx, label in enumerate(sorted(np.unique(metadata['dx'])))}
data_file = {ID : {'image' : None, 'label' : None} for ID in metadata['image_id']}

for ID in metadata['image_id']:
    data_file[ID]['image'] = toPath(ham_path, ID)
    data_file[ID]['label'] = toLabel(metadata[metadata['image_id'] == ID]['dx'].values[0], label_mapping)

# get class distribution
class_counts = {idx : 0 for idx in range(len(np.unique(metadata['dx'])))}
for ID in data_file.keys():
    class_counts[data_file[ID]['label']] += 1
    
ncls = len(class_counts.keys())
weights = torch.tensor([1 / val for val in class_counts.values()])*1000

In [ ]:
print("HAM 10000 Metadata")
print('------------------------------------------------------------------')
print(headers, '\n')
print("Number of Classes")
print(ncls, '\n')
print('Unique Labels')
print(label_mapping, '\n')
print("Class Balance")
print(class_counts, '\n')
print("Inverse Weighting")
print(weights)

In [ ]:
if COMPUTE_DISTRIBUTION: 
    ys, xs = [], []
    for ID in data_file.keys():
        img = cv.imread(data_file[ID]['image'])
        y, x, _ = img.shape
        ys.append(y)
        xs.append(x)
    
    print("Dataset Image Size Distribution")
    print("Num Patients", len(ys), len(xs))
    print("Unique Values", np.unique(ys), np.unique(xs))
    plt.scatter(y,x)
    plt.title("Image dimensions")
    plt.show()

## Experiment, Dataset, Dataloader

### Augmentations

In [ ]:
train_augmentations = process.Transformer()
train_augmentations.add(k.RandomHorizontalFlip())
train_augmentations.add(k.RandomVerticalFlip())
train_augmentations.add(k.RandomRotation(15))
train_augmentations.add(k.ColorJitter(0.1, 0.1, 0.1))
train_augmentations.add(txf.Lambda(lambda x: x.squeeze()))
train_txf = train_augmentations.transforms()

Since the dataset is imbalanced, partition train/validation/test by number of classes and then wrap in an oversampler

In [ ]:
# generate partitioned exp
partitioned_data = process.generateExperiment(data_file, ncls)
merged_data = process.restructureExperiments(partitioned_data)

# generate datasets
train_set = process.SkinSet(merged_data['train'])
val_set = process.SkinSet(merged_data['validation'])
test_set = process.SkinSet(merged_data['test'])

# Loaders
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)

# Model

Randomly initialized ResNet-50/18/34 or pretrained. Dense attempted but runtime is too slow :(

In [ ]:
def init_weights(m):
        if (type(m) == nn.Conv2d or type(m) == nn.Linear):
            nn.init.kaiming_normal_(m.weight)

if MODEL_ZOO: 
    
    def generateModel(n_cls, model_type, init=None, device=None):
        if model_type == 'RES':
            model = torchvision.models.resnet18(pretrained=True)
            in_ftr  = model.fc.in_features
            model.fc = nn.Linear(in_ftr,n_cls,bias=True)
        
        elif model_type == 'DENSE':
            model = torchvision.models.densenet121(pretrained=True)
            model_lin = model.classifier.in_features
            model.classifier = nn.Linear(model_lin, n_cls)

        if init is not None:
            model.apply(init)

        if device is not None:
            model = model.to(device)

        return model

    model = generateModel(ncls, model_type=MODEL_TYPE, init=None, device=device)
    
else:
    basic_config = ([3, 32, 64, 128, 256], 1, True, 0.15, 7)
    model = models.BasicCNN(*basic_config).to(device)
    model.apply(init_weights)

# Optimization and Criteria

Optimizer: AdamW for super convergence and fast training

Scheduler: Step LR decay 

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_period, lr_decay)
criterion = nn.CrossEntropyLoss(weight=weights).to(device)

# Metrics and Aggregation

In [ ]:
# Metric storage
train_stats = metrics.Aggregator()
val_stats = metrics.Aggregator()

# add stats
train_stats.addStat('loss')
val_stats.addStat('loss')
train_stats.addStat('acc', accuracy_score)
val_stats.addStat('acc', accuracy_score)
train_stats.addStat('bal', balanced_accuracy_score)
val_stats.addStat('bal', balanced_accuracy_score)

# Training 

In [ ]:
best_stat = 0
best_model = None
for epoch in range(epochs):
    
    t = time.time()
    
    # train
    preds, labels, t_loss = trainer.train(model, criterion, optimizer, scheduler, train_loader, device, train_txf)
    train_stats.logStat('loss', (t_loss,))
    train_stats.logStat('acc', (labels, preds))
    train_stats.logStat('bal', (labels, preds))
    
    # validate
    preds, labels, v_loss = trainer.evaluate(model, criterion, optimizer, scheduler, val_loader, device)
    val_stats.logStat('loss', (v_loss,))
    val_stats.logStat('acc', (labels, preds))
    val_stats.logStat('bal', (labels, preds))
    
    if val_stats.getStats('bal')[-1] > best_stat:
        best_stat = val_stats.getStats('bal')[-1]
        best_model = model.state_dict()
    
    t = time.time() - t
        
    print("Epoch:", epoch+1)
    print("--------------------------------")
    print("Time:", t)
    print("Training Loss:       ", t_loss)
    print("Validation Loss:     ", v_loss)
    print("Training Accuracy:   ", train_stats.getStats('acc')[-1])
    print("Validation Accuracy: ", val_stats.getStats('acc')[-1])
    print("Training Balanced:   ", train_stats.getStats('bal')[-1])
    print("Validation Balanced: ", val_stats.getStats('bal')[-1], '\n')
    
    
metrics.Plotter.plot(train_stats.getStats('loss'), val_stats.getStats('loss'), 'Epochs', 'Loss', 'Losses')
metrics.Plotter.plot(train_stats.getStats('acc'), val_stats.getStats('acc'), 'Epochs', 'Accuracy', 'Accuracies')
metrics.Plotter.plot(train_stats.getStats('bal'), val_stats.getStats('bal'), 'Epochs', 'Accuracy', 'Balanced Accuracies')

In [ ]:
if not os.path.exists('./balanced_ensembles'):
    os.mkdir('./balanced_ensembles')

In [ ]:
with open('balanced_exp.pickle', 'wb') as handle:
    pickle.dump(merged_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
torch.save(best_model, os.path.join('./balanced_ensembles', 'model_' + str(0) + '.pth.tar'))

# Testing

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
model.load_state_dict(torch.load(os.path.join('./balanced_ensembles', 'model_' + str(0) + '.pth.tar')))
preds, labels, _ = trainer.evaluate(model, criterion, optimizer, scheduler, test_loader, device)
test_acc = accuracy_score(labels, preds)
test_bal = balanced_accuracy_score(labels, preds)
conf_mat = confusion_matrix(labels, preds)
print("Test Accuracy: ", test_acc)
print("Test Balance: ", test_bal)
print(conf_mat)

# Ensemble 

In [ ]:
n_ensembles = 15

In [ ]:
for idx in range(n_ensembles):
    model = generateModel(ncls, MODEL_TYPE, None, device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_period, lr_decay)
    criterion = nn.CrossEntropyLoss(weight=weights).to(device)
    train_stats = metrics.Aggregator()
    val_stats = metrics.Aggregator()
    train_stats.addStat('loss')
    val_stats.addStat('loss')
    train_stats.addStat('acc', accuracy_score)
    val_stats.addStat('acc', accuracy_score)
    train_stats.addStat('bal', balanced_accuracy_score)
    val_stats.addStat('bal', balanced_accuracy_score)
    
    best_stat = 0
    best_model = None
    for epoch in range(epochs):

        t = time.time()

        # train
        preds, labels, t_loss = trainer.train(model, criterion, optimizer, scheduler, train_loader, device, train_txf)
        train_stats.logStat('loss', (t_loss,))
        train_stats.logStat('acc', (labels, preds))
        train_stats.logStat('bal', (labels, preds))

        # validate
        preds, labels, v_loss = trainer.evaluate(model, criterion, optimizer, scheduler, val_loader, device)
        val_stats.logStat('loss', (v_loss,))
        val_stats.logStat('acc', (labels, preds))
        val_stats.logStat('bal', (labels, preds))

        if val_stats.getStats('bal')[-1] > best_stat:
            best_stat = val_stats.getStats('bal')[-1]
            best_model = model.state_dict()

        t = time.time() - t
        
        print("MODEL:", idx+1)
        print("Epoch:", epoch+1)
        print("--------------------------------")
        print("Time:", t)
        print("Training Loss:       ", t_loss)
        print("Validation Loss:     ", v_loss)
        print("Training Accuracy:   ", train_stats.getStats('acc')[-1])
        print("Validation Accuracy: ", val_stats.getStats('acc')[-1])
        print("Training Balanced:   ", train_stats.getStats('bal')[-1])
        print("Validation Balanced: ", val_stats.getStats('bal')[-1], '\n')
        
    torch.save(best_model, os.path.join('./balanced_ensembles', 'model_' + str(idx+1) + '.pth.tar'))                               